# loop on the OCs to estimate isochrone and mass

In [1]:
using PyCall

## PopStar modules...
synthetic= pyimport("popstar.synthetic")
evolution= pyimport("popstar.evolution")
atmospheres= pyimport("popstar.atmospheres")
reddening= pyimport("popstar.reddening")

using Distributions, Statistics, Distances
using DataFrames
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = "/home/stephane/Science/cluster/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/plotdens"
ocdir= "$wdir/oc2"
isodir= "$wdir/isochrones"

cd(wdir)

In [2]:
cd(ocdir)
files= glob("*-oc.csv")
cd(wdir)

sample= rootdir*"/master/notebooks/data/BrowseTargets.18292.1530479692.fake"
df1= CSV.read(sample, delim= "|")
df1.name= strip.(df1.name)

noc= 0
for f in files
    fname= f[1:end-4] ; fvot= f[1:end-7] 
    ocfile= "$ocdir/$fname.csv" ; voname= "$fvot.vot"
    ocmassfile= "$ocdir/$fname-mass.csv"
    
    df= CSV.read(ocfile , delim= ";")
    
    t= split(fvot,"-")
    if length(t) == 3
        t[1]= t[1]*"-"*t[2]
    end
    ocname= t[1]
    
    println("## Name: $fname")
    logage= df1[(df1.name .== ocname), :log_age]
    # println(logage[1])
    
    ################################
    ## ISOCHRONES ##################
    ################################
    agmedian= median(df.ag[.! isnan.(df.ag)])
    distance= median(df.distance)
    nstar= length(df.distance)
    
    println("## N(star): $nstar")
    # Define isochrone parameters
    logAge= logage[1]  # Age in log(years)
    AKs= 0.0 # extinction in mags
    dist= distance # distance in parsec
    metallicity= 0.0 # Metallicity in [M/H]
    
    println("### log Age: $logAge")
    println("### AKs: $AKs")
    println("### distance: $dist")
    println("### metallicity: $metallicity")
    
    ## Define evolution/atmosphere models and extinction law
    if logAge>6.5
        evo_model = evolution.MergedBaraffePisaEkstromParsec() 
    else
        evo_model= evolution.MISTv1()
    end
    
    atm_func = atmospheres.get_merged_atmosphere
    red_law = reddening.RedLawHosek18b()

    ## Filter
    filt_list = [ "gaia,dr2_rev,Gbp", "gaia,dr2_rev,G"  ,"gaia,dr2_rev,Grp"]
    
    #####################################
    println("### Computing isochrones..")
    iso = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list, iso_dir= isodir)
    
    println("### Updating the DF and creating the new one with stellar mass..")
    
    data= iso.points
    println(data.keys())
    ndata= size(data)[1]

    mass= zeros(ndata)
    Gmag= zeros(ndata)
    BPmag= zeros(ndata)
    RPmag= zeros(ndata)
    color= zeros(ndata)

    for i in 1:ndata
        mass[i]= data[i][4]
        Gmag[i]= data[i][10]
        BPmag[i]= data[i][9]
        RPmag[i]= data[i][11]
        color[i]= BPmag[i]-RPmag[i]
    end

    ## Cluster
    BmR= df.bp .- df.rp
    GMAG= df.gbar + 5 .* log10.(df.distance) .- 17.
    

    ### Stellar mass 
    ################

    
    nxy= length(mass)
    A= Array{Float64}(undef,2,nxy)
    for i in 1:nxy
        A[1,i]= color[i]
        A[2,i]= Gmag[i]
    end
    
    d= Distances.Euclidean()
    mstar= zeros(length(BmR))

    for i in 1:length(BmR)
        starm= [BmR[i]; GMAG[i]]
        r= Distances.colwise(d, A, starm)
    
        idx= argmin(r)
        mstar[i]= mass[idx]
    end
    
    df.mass= mstar
    
    ## Some Logs mass stats...
    massmean= mean(mstar)
    println("### Mass (mean): $massmean")
    
    ### save the OC df
    CSV.write(ocmassfile, df, delim=";") 
    println("\n\n")
end
      

## Name: ASCC 10-2.7deg-oc
## N(star): 119
### log Age: 8.717
### AKs: 0.0
### distance: 661.0905129409848
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]


┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:78
└ @ Core ./In[2]:78
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:78
└ @ Core ./In[2]:78
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:79
└ @ Core ./In[2]:79
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:79
└ @ Core ./In[2]:79
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:80
└ @ Core ./In[2]:80
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instead.
│   caller = top-level scope at In[2]:80
└ @ Core ./In[2]:80
┌ Warning: `getindex(o::PyObject, i::Integer)` is deprecated, use `get(o, i - 1)` instea

### Mass (mean): 1.0398919579831933



## Name: ASCC 100-1.8deg-oc
## N(star): 149
### log Age: 7.95
### AKs: 0.0
### distance: 359.22359249347744
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.6322624161073825



## Name: ASCC 101-2.5deg-oc
## N(star): 55
### log Age: 8.62
### AKs: 0.0
### distance: 397.7731682887633
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.0345083636363637



## Name: ASCC 103-1.4deg-oc
## N(star): 104
### log Age: 8.39
### AKs: 0.0
### distance: 500.99039969178705
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff"

Starting filter: gaia,dr2_rev,Grp   Elapsed time: 36.87 seconds### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.1446733333333332



## Name: ASCC 114-1.0deg-oc
## N(star): 159
### log Age: 7.75
### AKs: 0.0
### distance: 944.140687558289
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.077652830188679



## Name: ASCC 115-1.1deg-oc
## N(star): 72
### log Age: 8.35
### AKs: 0.0
### distance: 426.5592747602284
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.6536542222222221



## 

M =   9.671 Msun  T =  9417 K  m_hst_Gbp = 5.28### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.3496248120300751



## Name: ASCC 122-2.8deg-oc
## N(star): 433
### log Age: 7.7
### AKs: 0.0
### distance: 498.4601520091302
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.7141773672055429



## Name: ASCC 124-1.6deg-oc
## N(star): 91
### log Age: 8.195
### AKs: 0.0
### distance: 716.2381821863676
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.2217169340659342



## Name: ASCC 125

...

M =   1.787 Msun  T =  4574 K  m_hst_Gbp = 10.31### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.0937880608108106



## Name: IC 4665-3.5deg-oc
## N(star): 93
### log Age: 7.63
### AKs: 0.0
### distance: 344.80922004190694
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.1049107526881718



## Name: IC 4756-4.0deg-oc
## N(star): 355
### log Age: 8.79
### AKs: 0.0
### distance: 477.05758155794996
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.0998459549295776



## Name: IC 4996

M =   0.090 Msun  T =  2323 K  m_hst_Grp = 23.78### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 0.8975010372093024



## Name: Kronberger 1-1.0deg-oc
## N(star): 202
### log Age: 6.9
### AKs: 0.0
### distance: 1131.0800446242088
### metallicity: 0.0
### Computing isochrones..

M =   2.148 Msun  T =  5830 K  m_hst_Grp = 9.46
      Time taken: 8.78 seconds
Changing to logg=3.50 for T= 28288 logg=3.49
Changing to logg=3.50 for T= 28223 logg=3.48
Changing to logg=3.50 for T= 27913 logg=3.44
Changing to logg=3.50 for T= 27599 logg=3.39
Changing to logg=3.50 for T= 27290 logg=3.33
Changing to logg=3.50 for T= 27208 logg=3.32
Changing to logg=3.50 for T= 27171 logg=3.29
Changing to logg=3.50 for T= 27133 logg=3.26
Changing to logg=3.50 for T= 27152 logg=3.26
Changing to logg=3.50 for T= 27171 logg=3.26
Changing to logg=3.50 for T= 27189 logg=3.27
Changing 

Changing to logg=4.00 for T= 35843 logg=3.66### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.3207618811881185



## Name: Kronberger 72-1.0deg-oc
## N(star): 88
### log Age: 8.85
### AKs: 0.0
### distance: 864.8137486571497
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.058161909090909



## Name: Kronberger 84-1.0deg-oc
## N(star): 126
### log Age: 8.825
### AKs: 0.0
### distance: 939.1571645100696
### metallicity: 0.0
### Computing isochrones..
### Updating the DF and creating the new one with stellar mass..
["L", "Teff", "R", "mass", "logg", "isWR", "mass_current", "phase", "m_hst_Gbp", "m_hst_G", "m_hst_Grp"]
### Mass (mean): 1.0695949285714292



## Name: L

Excessive output truncated after 529045 bytes.





## Name: Loden 1256-1.0deg-oc
## N(star): 117
### log Age: 8.58
### AKs: 0.0
### distance: 969.7124511480021
### metallicity: 0.0
### Computing isochrones..